In [1]:
### Benjamin Tollison ###
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sympy as sp
from IPython.display import Latex, Math, display
from sympy import (
    Eq,
    Function,
    Matrix,
    cos,
    cosh,
    exp,
    integrate,
    lambdify,
    pi,
    sin,
    sinh,
    symbols,
)
from decimal import Decimal
from sympy.solvers.pde import pdsolve
from sympy.solvers.solveset import linsolve
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))
def format_scientific(number:float):
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]
deg2rad = np.pi/180
rad2deg = 180/np.pi

In [6]:
area_throat = 0.042
area_exit = 4.91
radius_throat = (area_throat/np.pi)**0.5
radius_exit = (area_exit/np.pi)**0.5
displayVariableWithUnits('r_{throat}',radius_throat,'m')
displayVariableWithUnits('r_{exit}',radius_exit,'m')


<IPython.core.display.Math object>

<IPython.core.display.Math object>

4.914000000000001


### Here is how I can find all the mach values from the area equation
$$\frac{\sigma}{\sigma_{cr}} = 
\frac{1}{M} \left[\frac{2}{\gamma+1}\left(1+\frac{\gamma-1}{2}M^2\right)\right]^\frac{\gamma+1}{2(\gamma-1)}$$
We can us Householder's second order root method from the nasa report:\
https://www.grc.nasa.gov/www/winddocs/utilities/b4wind_guide/mach.html
$$ x_{i+1} = x_i - \frac{2f}{f'-\sqrt{f'^2-ff''}}$$
With the subsonic regime being defined by:
$$f = (P+QX)^\frac{1}{Q} - RX = 0$$
$$f' = (P+QX)^{\frac{1}{Q}-1} - R$$
$$f'' = P(P+QX)^{\frac{1}{Q}-2}$$
and the supersonic regime can be found with the following:
$$f = (PX+Q)^\frac{1}{P} - RX = 0$$
$$f' = (PX+Q)^{\frac{1}{P}-1} - R$$
$$f'' = Q(PX+Q)^{\frac{1}{P}-2}$$
And the Coefficients can be defined by:
$$P=\frac{2}{\gamma+1}$$
$$Q = \frac{\gamma-1}{\gamma+1}=1-P $$
$$X_{subsonic} = M^2$$
$$X_{supersonic} = \frac{1}{M^2}$$
$$R_{subsonic} = \left(\frac{\sigma}{\sigma_{cr}}\right)^2$$
$$R_{supersonic} = \left(\frac{\sigma}{\sigma_{cr}}\right)^\frac{2Q}{P}$$
Therefore we can have 4 possible solutions. Subsonic->subsonic, subsonic->supersonic, supersonic->subsonic, or supersonic->supersonic

In [8]:
def HouseholdP2(x_intial:float,scheme_function,scheme_prime,scheme_double_prime)->float:
  max_iterations = 1000
  while abs(scheme_function(x_intial)) > 1e-8:
    x_intial = x_intial - ((2*scheme_function(x_intial))/(scheme_prime(x_intial) - (scheme_prime(x_intial)**2-scheme_function(x_intial)*scheme_double_prime(x_intial))**0.5))
    max_iterations -=1
    if max_iterations ==0:
      print('The scheme didn\'t converge')
      break
  return x_intial
def Householder(x_position:float,section_supersonic:bool,area_function)->float:
  P = 2/2.4
  Q = 1-P
  if section_supersonic==False:
    R = (area_function(x_position))**2
    a = P**(1/Q)
    r = (R-1)/(2*a)
    x_intial = 1 / ((1+r)+np.sqrt(r*(r+2)))
    f = lambda X : (P+Q*X)**(1/Q) - R*X
    f_prime = lambda X: (P+Q*X)**((1/Q)-1) - R
    f_double_prime = lambda X: P*(P+Q*X)**((1/Q)-2)
    x_final = HouseholdP2(x_intial,f,f_prime,f_double_prime)
    return (x_final)**0.5
  if section_supersonic == True:
    R = (area_function(x_position))**(2*Q/P)
    a = Q**(1/P)
    r = (R-1)/(2*a)
    x_intial = 1 / ((1+r)+np.sqrt(r*(r+2)))
    f = lambda X : (P*X+Q)**(1/P) - R*X
    f_prime = lambda X: (P*X+Q)**((1/P)-1) - R
    f_double_prime = lambda X: Q*(P*X+Q)**((1/P)-2)
    x_final = abs(HouseholdP2(x_intial,f,f_prime,f_double_prime))
    return 1/(x_final)**0.5
original_geometry = lambda x:2-np.cos(np.pi*x)
mach_values = {'SubsonicOnly':[],'Subsonic/Supersonic':[],'Supersonic/Subsonic':[],'SupersonicOnly':[]}
area_ratio = {'Area/Area_cr':[]}
x_values_section_1 = np.linspace(-0.25,0,200)
x_values_section_2 = np.linspace(0,1,800)
area_ratio['Area/Area_cr'] = [original_geometry(x) for x in x_values_section_1] + [2-np.cos(np.pi*x) for x in x_values_section_2]
mach_values['SubsonicOnly'] = [Householder(x,False,original_geometry)for x in x_values_section_1] + [Householder(x,False,original_geometry)for x in x_values_section_2]
mach_values['Subsonic/Supersonic'] = [Householder(x,False,original_geometry)for x in x_values_section_1] + [Householder(x,True,original_geometry)for x in x_values_section_2]
mach_values['Supersonic/Subsonic'] = [Householder(x,True,original_geometry)for x in x_values_section_1] + [Householder(x,False,original_geometry)for x in x_values_section_2]
mach_values['SupersonicOnly'] = [Householder(x,True,original_geometry)for x in x_values_section_1] + [Householder(x,True,original_geometry)for x in x_values_section_2]